In [18]:
import pandas as pd
from neo4j import GraphDatabase

pd.options.display.max_rows = 200
pd.options.display.max_columns = 500

## Establish the connection to our Sandbox

The following class will handle the connection and queries for us.  We will need to get the IP address of our Sandbox instance as well as the password and provide these in the subsequent cell.

Prior to running this notebook you should have populated the database using the queries in `./cypher_queries/populate_db.cql`.

In [2]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [4]:
uri = ''
pwd = ''

conn = Neo4jConnection(uri=uri, user="neo4j", pwd=pwd)
conn.query('MATCH (n) RETURN COUNT(n) AS count')

[<Record count=8627>]

## Creating a graph projection



In [6]:
projection_query = """CALL gds.graph.create(
                      'routes',
                      'Airport',
                      'HAS_ROUTE'
                      )
                      YIELD
                          graphName, nodeProjection, nodeCount, relationshipProjection, relationshipCount
                    """

conn.query(projection_query)

[<Record graphName='routes' nodeProjection={'Airport': {'label': 'Airport', 'properties': {}}} nodeCount=3503 relationshipProjection={'HAS_ROUTE': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'HAS_ROUTE', 'properties': {}}} relationshipCount=46389>]

## PageRank

Recall how this was calculated in SQL???

In [7]:
pagerank_query = """CALL gds.pageRank.stream('routes')
                    YIELD nodeId, score
                    RETURN gds.util.asNode(nodeId).iata AS iata, 
                        gds.util.asNode(nodeId).descr AS description, score
                    ORDER BY score DESC, iata ASC
                    """

pr_df = pd.DataFrame([dict(_) for _ in conn.query(pagerank_query)])
pr_df.head()

,iata,description,score
0,DFW,Dallas/Fort Worth International Airport,11.979783
1,ORD,Chicago O'Hare International Airport,11.162988
2,DEN,Denver International Airport,10.997299
3,ATL,Hartsfield - Jackson Atlanta International Air...,10.389948
4,IST,Istanbul International Airport,8.425801


In [10]:
pagerank_write_query = """CALL gds.pageRank.write('routes', 
                            {
                                writeProperty: 'pagerank'
                            }
                          )
                          YIELD nodePropertiesWritten, ranIterations
                          """

conn.query(pagerank_write_query)

[<Record nodePropertiesWritten=3503 ranIterations=20>]

## Path finding (Dijkstra's algorithm for shortest path)

Remember that whole packages (ex: pgRouting) exist to do this in SQL.  Why?  Because it is not trivial!

In [13]:
weighted_proj_query = """CALL gds.graph.create(
                              'routes-weighted',
                              'Airport',
                              'HAS_ROUTE',
                                  {
                                      relationshipProperties: 'distance'
                                  }
                          )
                      """

conn.query(weighted_proj_query)

[<Record nodeProjection={'Airport': {'label': 'Airport', 'properties': {}}} relationshipProjection={'HAS_ROUTE': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'HAS_ROUTE', 'properties': {'distance': {'defaultValue': None, 'property': 'distance', 'aggregation': 'DEFAULT'}}}} graphName='routes-weighted' nodeCount=3503 relationshipCount=46389 createMillis=170>]

In [14]:
sp_query = """MATCH (source:Airport {iata: 'DEN'}), (target:Airport {iata: 'MLE'})
              CALL gds.shortestPath.dijkstra.stream('routes-weighted', {
                  sourceNode: source,
                  targetNode: target,
                  relationshipWeightProperty: 'distance'
              })
              YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
              RETURN
                  index,
                  gds.util.asNode(sourceNode).iata AS sourceNodeName,
                  gds.util.asNode(targetNode).iata AS targetNodeName,
                  totalCost,
                  [nodeId IN nodeIds | gds.util.asNode(nodeId).iata] AS nodeNames,
                  costs,
                  nodes(path) as path
              ORDER BY index
              """

sp_df = pd.DataFrame([dict(_) for _ in conn.query(sp_query)])
sp_df.head(10)

,index,sourceNodeName,targetNodeName,totalCost,nodeNames,costs,path
0,0,DEN,MLE,9704.0,"[DEN, KEF, HEL, VKO, MLE]","[0.0, 3556.0, 5074.0, 5629.0, 9704.0]","[(altitude, descr, longest, iata, city, icao, ..."


## Clustering (Louvain)

In [19]:
louvain_query = """CALL gds.louvain.stream('routes')
                   YIELD nodeId, communityId
                   RETURN 
                       communityId,
                       SIZE(COLLECT(gds.util.asNode(nodeId).iata)) AS number_of_airports,
                       COLLECT(gds.util.asNode(nodeId).city) AS city
                   ORDER BY number_of_airports DESC, communityId
                   """

louvain_df = pd.DataFrame([dict(_) for _ in conn.query(louvain_query)])
louvain_df.head()

,communityId,number_of_airports,city
0,3282,695,"[Atlanta, Anchorage, Austin, Nashville, Boston..."
1,2294,497,"[London, London, Paris, Frankfurt, Helsinki, D..."
2,2784,418,"[Dubai, New Delhi, Mumbai, Doha, Calicut, Hyde..."
3,2980,254,"[Tokyo, Singapore, Melbourne, Hong Kong, Beiji..."
4,3260,181,"[Sydney, Perth, Auckland, Wellington, Brisbane..."


## Node similarity

In [21]:
ns_query = """CALL gds.nodeSimilarity.stream(
                   'routes',
                   {
                       topK: 3
                   }
               )
               YIELD node1, node2, similarity
               RETURN 
                   gds.util.asNode(node1).city AS City1, 
                   COLLECT(gds.util.asNode(node2).city) AS City2, 
                   COLLECT(similarity) AS similarity
               ORDER BY similarity[0] DESC
               """

ns_df = pd.DataFrame([dict(_) for _ in conn.query(ns_query)])
ns_df.head(10)

,City1,City2,similarity
0,Santa Fe,"[Flagstaff, Durango, Yuma]","[1.0, 1.0, 0.75]"
1,Manhattan,"[Sioux City, Champaign/Urbana, Columbia]","[1.0, 0.6666666666666666, 0.6666666666666666]"
2,Monroe,"[Fort Hood/Killeen, Alexandria, Fort Smith]","[1.0, 1.0, 0.6666666666666666]"
3,San Angelo,"[Abilene, Waco, Texarkana]","[1.0, 1.0, 1.0]"
4,Wichita Falls,"[Abilene, Waco, Texarkana]","[1.0, 1.0, 1.0]"
5,Tyler,"[Lake Charles, College Station, Brownsville]","[1.0, 1.0, 0.6666666666666666]"
6,Burlington,"[Quincy, Decatur, Cape Girardeau, Greensboro, ...","[1.0, 1.0, 0.6666666666666666, 0.6666666666666..."
7,Florence,"[Greenville, Lynchburg, Staunton/Waynesboro/Ha...","[1.0, 1.0, 0.5]"
8,Topeka,"[Hancock, Dubuque, Waterloo]","[1.0, 1.0, 1.0]"
9,Marquette,"[La Crosse, Mosinee, Saginaw]","[1.0, 1.0, 0.75]"
